In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
import csv
import scipy
import astropy

def merge_windows(windows):
    # Sort the windows based on the start time
    windows.sort(key=lambda x: x[0])

    merged_windows = [windows[0]]

    for current_start, current_end in windows[1:]:
        last_merged_start, last_merged_end = merged_windows[-1]

        # Check if the current window overlaps with the last merged window
        if current_start <= last_merged_end:
            # Merge the windows by extending the end time of the last merged window
            merged_windows[-1] = (last_merged_start, max(last_merged_end, current_end))
        else:
            # No overlap, add the current window to the merged list
            merged_windows.append((current_start, current_end))
                
    return merged_windows

from astropy.timeseries import LombScargle
def remove_detected_windows(time_array, data_array, detected_windows):
    """
    Removes data corresponding to abnormal windows from the time and observation arrays.
    
    Parameters:
        time_array (np.ndarray): Array of time values.
        data_array (np.ndarray): Array of observation values.
        detected_windows (list of tuples): List of abnormal windows, each defined by a tuple (start_index, end_index).
    
    Returns:
        tuple: Filtered time_array and data_array with abnormal data removed.
    """
    if len(time_array) != len(data_array):
        raise ValueError("time_array and data_array must have the same length.")
    
    # Create a mask of valid data points
    valid_mask = np.ones(len(time_array), dtype=bool)
    
    # Mark indices in abnormal windows as invalid
    for start_idx, end_idx in detected_windows:
        valid_mask[start_idx:end_idx + 1] = False
    
    # Apply the mask to both arrays
    filtered_time = time_array[valid_mask]
    filtered_observation = data_array[valid_mask]
    
    return filtered_time, filtered_observation

In [2]:
def plot_seismogram_with_gaps(filtered_t, filtered_seismogram, gap_threshold=6.625):
    # Detect gaps based on the filtered time array
    segments = []
    current_segment = [0]

    for i in range(1, len(filtered_t)):
        if filtered_t[i] - filtered_t[i - 1] > gap_threshold:
            current_segment.append(i - 1)
            segments.append(current_segment)
            current_segment = [i]
    current_segment.append(len(filtered_t) - 1)
    segments.append(current_segment)

    for segment in segments:
        start, end = segment
        plt.plot(
            filtered_t[start:end + 1],
            filtered_seismogram[start:end + 1],
            color='k'
        )
    return

In [3]:
# Representing long-period seismograms as a sum of decaying cosinusoids ()
def get_simulated_seismogram(
    mode_num = 7,
    amplitude_list = [1, 1, 1, 1, 1, 1, 1],
    period_list = [100, 200, 300, 400, 500, 600, 700],
    phase_list = [0, 0, 0, 0, 0, 0, 0],
    quality_factor_list = [1500, 1500, 1500, 1500, 1500, 1500, 1500],
    duration = 120000,
    dt = 1.0/6.625
    ):
    """
    mode_num: number of modes to simulate
    amplitude_list: list of amplitudes for each mode
    period_list: list of periods for each mode
    phase_list: list of phases for each mode
    quality_factor_list: Q for each mode
    duration: duration of the seismogram in seconds
    dt: time step in seconds
    """
    t = np.arange(0, duration, dt)
    seismogram = np.zeros_like(t)
    freq_list = 1.0/np.array(period_list)
    for i in range(mode_num):
        attenuation_factor = 2*np.pi*freq_list[i]/quality_factor_list[i]
        seismogram += amplitude_list[i]*np.exp(-attenuation_factor*t)*np.cos(2*np.pi*freq_list[i]*t + phase_list[i])
    
    return t, seismogram

In [4]:
# MW setting with 2 overtones
mode_num = 27
amplitude_list = np.ones(mode_num)
# period list from Gudkova and Raevskii (2013) based on the model from Weber et al. (2011).
period_list = [16.15*60.0, 10.47*60.0, 8.163*60.0, 6.812*60.0, 5.883*60.0, 5.192*60.0, 4.655*60.0, 4.224*60.0, 3.870*60.0,
               8.950*60.0, 6.581*60.0, 5.239*60.0, 4.378*60.0, 3.776*60.0, 3.332*60.0, 2.992*60.0, 2.723*60.0, 2.506*60.0,
               5.153*60.0, 4.950*60.0, 3.926*60.0, 3.376*60.0, 3.033*60.0, 2.771*60.0, 2.554*60.0, 2.369*60.0, 2.208*60.0]

phase_list = np.zeros(mode_num)
quality_factor_list = np.ones(mode_num) * 1500
duration = 40*3600
dt = 1.0/6.625
t, seismogram = get_simulated_seismogram(mode_num, amplitude_list, period_list, phase_list, quality_factor_list, duration, dt)

In [5]:
# define a function to calculate the minus_fft when given a sequence and a list of windows
def minus_fft_calculator(sequence, windows, cosine_taper_length=50):
    """
    sequence: the sequence of the signal
    windows: a list of windows which has the start and end point of the window
    cosine_taper_length: the length of the cosine taper
    """
    
    taper = np.cos(np.linspace(0, np.pi, int(cosine_taper_length*2)))
    window_summation = np.zeros(len(sequence))
    for window in windows:
        start_dx = window[0]
        end_dx = window[1]
        temp_window = np.zeros(len(sequence))
        temp_window[start_dx:end_dx] = 1
        # make the edge smooth with a cosine window
        #print(start_dx, end_dx, cosine_taper_length, len(temp_window), len(taper))
        temp_window[start_dx-cosine_taper_length:start_dx] = temp_window[start_dx-cosine_taper_length:start_dx]*taper[0:cosine_taper_length]


        temp_window[end_dx:end_dx+cosine_taper_length] = temp_window[end_dx:end_dx+cosine_taper_length]*taper[cosine_taper_length:]

        window_summation = window_summation + temp_window
    # apply the window to the signal
    sequence_windowed = sequence*window_summation

    # apply hann window to the windowed signal
    sequence_windowed = sequence_windowed*scipy.signal.windows.hann(len(sequence))
    sequence_windowed_fft = np.fft.fft(sequence_windowed)
    # apply hann window to the original signal
    sequence = sequence*scipy.signal.windows.hann(len(sequence))

    sequence_fft = np.fft.fft(sequence)

    minus_fft = sequence_fft - sequence_windowed_fft
    
    return minus_fft

In [6]:
def plot_waveform_and_spectrum_with_disturbances_removed(t, dt, seismogram, polluted_seismogram, disturbance_windows, freqmin=1*1e-3, freqmax=12*1e-3, window='hanning', pollute_rate = None):
    """
    t: time array
    dt: time step
    seismogram: seismogram array
    """
    n = len(seismogram)
    f = np.fft.fftfreq(n, dt)
    f = f[:n//2]

    if window == 'hanning':
        windowed_seismogram = seismogram * np.hanning(n)
    else:
        windowed_seismogram = seismogram
    
    spectrum = np.abs(np.fft.fft(windowed_seismogram))[:n//2]
    plt.figure(figsize=(20, 26))
    plt.subplot(8, 1, 1)
    plt.plot(t, windowed_seismogram, color='k')
    plt.xlabel('Time (s)', fontsize=18)
    plt.ylabel('Amplitude', fontsize=18)
    #plt.title('Hanning windowed seismogram', fontsize=20)
    plt.text(0.5, 0.955, 'Hanning windowed seismogram', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
    plt.xlim(0, t[-1])
    plt.ylim(np.min(windowed_seismogram) -2, np.max(windowed_seismogram) + 2)
    plt.text(-0.05, 1.00, '(a)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.subplot(8, 1, 2)
    plt.plot(f, spectrum, color='k')
    plt.xlabel('Frequency (Hz)', fontsize=18)
    plt.ylabel('Amplitude', fontsize=18)
    plt.xlim(freqmin, freqmax)
    #plt.title('Hanning windowed spectrum', fontsize=20)
    plt.text(0.5, 0.955, 'Hanning windowed spectrum', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
    plt.text(-0.05, 1.00, '(b)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')
    for mode in period_list:
        plt.axvline(1.0/mode, color='orange', linestyle='--', lw=2, zorder=-1)
    plt.axvline(0, color='orange', linestyle='--', label='Eigenfrequencies',lw=2, zorder=-1)
    plt.legend(loc='upper right', fontsize=14)

    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.subplot(8, 1, 3)
    if window == 'hanning':
        windowed_polluted_seismogram = polluted_seismogram * np.hanning(n)
    else:
        windowed_polluted_seismogram = polluted_seismogram

    plt.plot(t, windowed_polluted_seismogram, color='k')
    plt.xlabel('Time (s)', fontsize=18)
    plt.ylabel('Amplitude', fontsize=18)
    #plt.title('Seismogram with {:.0f}% data polluted by disturbances'.format(pollute_rate*100), fontsize=20)
    plt.text(0.5, 0.955, 'Seismogram with {:.0f}% data polluted by disturbances'.format(pollute_rate*100), transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
    plt.text(-0.05, 1.00, '(c)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')

    for window in disturbance_windows:
        plt.axvspan(window[0]*dt, window[1]*dt, color='pink', alpha=0.5, ymin=0.2, ymax=0.8)
    # add a label for the disturbancees
    plt.axvspan(0, 0, color='pink', alpha=0.5, label='Disturbances')
    plt.legend(loc='upper right', fontsize=14)
    plt.xlim(0, t[-1])
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    spectrum_polluted = np.abs(np.fft.fft(windowed_polluted_seismogram))[:n//2]
    plt.subplot(8, 1, 4)
    plt.plot(f, spectrum_polluted, color='k')
    for mode in period_list:
        plt.axvline(1.0/mode, color='orange', linestyle='--', lw=2, zorder=-1)
    #plt.axvline(0, color='orange', linestyle='--', label='Eigenfrequencies',lw=2, zorder=-1)
    #splt.legend(loc='upper right', fontsize=14)
    plt.xlabel('Frequency (Hz)', fontsize=18)
    plt.ylabel('Amplitude', fontsize=18)
    plt.xlim(freqmin, freqmax)
    #plt.title('Polluted spectrum', fontsize=20)
    plt.text(0.5, 0.955, 'Polluted spectrum', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
    plt.text(-0.05, 1.00, '(d)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.subplot(8, 1, 5)
    zeromuted_seismogram = windowed_polluted_seismogram.copy()
    for window in disturbance_windows:
        zeromuted_seismogram[window[0]:window[1]] = 0
    minus_fft = minus_fft_calculator(zeromuted_seismogram, disturbance_windows)
    
    plt.plot(t, zeromuted_seismogram, color='k')
    """
    for window in disturbance_windows:
        plt.axvspan(window[0]*dt, window[1]*dt, color='pink', alpha=0.5)
    # add a label for the disturbancees
    plt.axvspan(0, 0, color='pink', alpha=0.5, label='disturbances')
    plt.legend()
    """
    plt.xlabel('Time (s)', fontsize=18)
    plt.ylabel('Amplitude', fontsize=18)
    #plt.title('Seismogram with disturbances muted', fontsize=20)
    plt.text(0.5, 0.955, 'Seismogram with disturbances muted', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
    plt.text(-0.05, 1.00, '(e)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')
    plt.xlim(0, t[-1])
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.subplot(8, 1, 6)
    plt.plot(f, np.abs(minus_fft)[:n//2], color='k')
    plt.xlabel('Frequency (Hz)', fontsize=18)
    plt.ylabel('Power', fontsize=18)
    plt.xlim(freqmin, freqmax)
    #plt.title('Spectrum of the muted data', fontsize=20)
    plt.text(0.5, 0.955, 'Spectrum of the muted data', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
    for mode in period_list:
        plt.axvline(1.0/mode, color='orange', linestyle='--', lw=2, zorder=-1)
    #plt.axvline(0, color='orange', linestyle='--', label='Eigenfrequencies',lw=2, zorder=-1)
    #plt.legend(loc='upper right', fontsize=14)

    filtered_t, filtered_seismogram = remove_detected_windows(t, polluted_seismogram, disturbance_windows)
    frequency, power = LombScargle(filtered_t, filtered_seismogram).autopower(method='fastchi2',minimum_frequency=1e-5,maximum_frequency=1e-2,samples_per_peak=200)
    plt.text(-0.05, 1.00, '(f)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.subplot(8, 1, 7)
    #plt.scatter(filtered_t, filtered_seismogram, color='k', s = 1)
    plot_seismogram_with_gaps(filtered_t, filtered_seismogram, gap_threshold=6.625)
    plt.xlabel('Time (s)', fontsize=18)
    plt.ylabel('Amplitude', fontsize=18)
    #plt.title('Gapped Seismogram', fontsize=20)
    plt.text(0.5, 0.955, 'Gapped Seismogram', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
    plt.xlim(0, t[-1])
    plt.text(-0.05, 1.00, '(g)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)

    plt.subplot(8, 1, 8)
    plt.plot(frequency, power, color='k')
    for mode in period_list:
        plt.axvline(1.0/mode, color='orange', linestyle='--', lw=2, zorder=-1)
    #plt.axvline(0, color='orange', linestyle='--', label='Eigenfrequencies',lw=2, zorder=-1)
    #plt.legend(loc='upper right', fontsize=14)
    plt.xlabel('Frequency (Hz)', fontsize=18)
    plt.ylabel('Power', fontsize=18)
    #plt.title('Lomb-Scargle Periodogram', fontsize=20)
    plt.text(0.5, 0.955, 'Lomb-Scargle Periodogram', transform=plt.gca().transAxes, fontsize=24, va='top', ha='center', backgroundcolor='white', zorder=10)
    plt.xlim(freqmin, freqmax)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.text(-0.05, 1.00, '(h)', transform=plt.gca().transAxes, fontsize=18, fontweight='bold', va='top', ha='right')
    plt.tight_layout()
    plt.subplots_adjust(left=0.08, right=0.980, top=0.98, bottom=0.03, hspace=0.3)
    plt.savefig('Figure_4_usage_pollute_rate_{:.2f}.png'.format(pollute_rate), dpi=600)
    #plt.show()
    plt.close()

In [7]:
disturbance_data = np.load('disturbance_data_for_random_sample_usage.npy', allow_pickle=True)

In [8]:
pollute_rate_list = [0.05, 0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95]

In [9]:
for pollute_rate in pollute_rate_list:
    polluted_seismogram = seismogram.copy()
    disturbance_windows = []
    np.random.seed(0)
    while True:
        # randomly select disturbancees until most of the seismogram is polluted to
        sampled_disturbance = np.random.choice(disturbance_data).copy().astype(np.float64)
        sampled_disturbance -= np.mean(sampled_disturbance)
        disturbance_start = np.random.randint(51, len(seismogram)-51)
        t_disturbance_len = len(sampled_disturbance)
        # if the disturbance is too long, we need to cut it to prevent it from exceeding the pollute rate
        if t_disturbance_len > 7000:
            sampled_disturbance = sampled_disturbance[:8000]
            t_disturbance_len = 7000
        try:
            polluted_seismogram[disturbance_start:disturbance_start+t_disturbance_len] += sampled_disturbance
        except:
            continue
        disturbance_windows.append([disturbance_start, disturbance_start+t_disturbance_len])

        disturbance_windows = merge_windows(disturbance_windows)
        # calculate the polluted rate
        cur_polluted_rate = sum([win[1]-win[0] for win in disturbance_windows])/len(seismogram)
        if cur_polluted_rate >= pollute_rate:
            break
    plot_waveform_and_spectrum_with_disturbances_removed(t, 1.0/6.625, seismogram, polluted_seismogram, disturbance_windows, window='hanning', freqmin=0.1*1e-3, freqmax=9*1e-3, pollute_rate = cur_polluted_rate)

c:\Users\77918\anaconda3\envs\obspy\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\77918\anaconda3\envs\obspy\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
